# Exploring, filtering, grouping and viewing data in [pandas](https://pandas.pydata.org/)
Demonstration notebook, using UK Police data on street-level crime in the London Metropolitan region, from https://data.police.uk/


In [ ]:
import pandas as pd
import seaborn as sns
#Set max row and max column for display in Jupyter Notebook
pd.set_option('display.max_row', 150)
pd.set_option('display.max_columns', 20)

In [ ]:
# Set Seaborn styling parameters for plots
sns.set(rc={'figure.figsize':(15,10)}) # Set figure size for all plots
sns.set_context("notebook") # Set how plots are displayed

# Read and inspect data
The sample data here is Metropolitan Police cases, in CSV format. We use `read_csv()` to load the file.

In [ ]:
# URL of sample data file on GitHub
file = r'https://github.com/conceptbin/workshops/raw/main/pandas_data_analysis/data/2023-03-metropolitan-street.csv'
# Create dataframe (df)
df = pd.read_csv(file)

## Dimensions of the dataframe

In [ ]:
# Show the top rows
df.head()

In [ ]:
# Size of the dataframe (count of rows and columns):
df.shape

## Handling empty values with isna() and dropna()

In [ ]:
# Summarise the empty values in df by column
df.isna().sum()

As we are interested in where incidents took place, we can remove all rows with blanks (NaN values) in the 'LSOA code' column.

In [ ]:
# Remove rows with empty values (NaN) in the 'LSOA name' column:
df.dropna(subset=['LSOA code'], inplace=True)

In [ ]:
#Check if the NaN values have been dropped from df
df.isna().sum()

In [ ]:
df.info()

## Show head, tail or slice
`df.head()`, last rows:` df.tail()`. Show slice by index number: `df[5:10]` (shows rows 5-10). 

For large dataframes, if you just run a cell with the name of the dataframe, it will display the head and tail, omitting the middle.

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
#Show a slice of the rows by index
df[200:205]

## value_counts()
For categorical data, show the number of occurrences. In this case, number of incidents for each 'Crime type' category:

In [ ]:
df['Crime type'].value_counts()

## info()

In [ ]:
df.info()

# Selecting and filtering data

Select only columns containing categorical data ('object' type) with `select_dtypes()` and summarise with `describe()`

In [ ]:
(df.select_dtypes(include=['object'])).describe()

## Slice string
Create a new column with the Borough name, so that we can select rows by borough. The `slice()` function returns a defined segment of the text (or *string*) in the "LSOA name" column.

In [ ]:
#Create a Borough-column:
df['Borough'] = df['LSOA name'].str.slice(0, -5)

## List unique values by column

In [ ]:
df['Borough'].unique()

Now that we know that the dataset includes incidents that took place outside of London, let's select only the London Boroughs in the dataset (just as we would use a filter in Excel).

## Select with contains()
Select with a keyword. Good for single-keyword filtering, or stringing conditions together with & or | ('or')

In [ ]:
#Create separate dataframe for selected borough using part of the name
df_select = df[df['LSOA name'].str.contains('Leeds')]

In [ ]:
df_select

In [ ]:
#Check the shape of df_select. This shows us how many cases (rows) the borough has
df_select.shape

## Select with isin()
Select rows using a list of values.

In [ ]:
#List of London Boroughs, plus City of London
LB_list = ['Barking and Dagenham', 'Barnet', 'Bexley', 'Brent', 'Bromley','Camden', 'City of London', 'Croydon', 'Ealing', 'Enfield', 'Greenwich', 'Hackney', 'Hammersmith and Fulham', 'Haringey',
       'Harrow', 'Havering', 'Hillingdon', 'Hounslow', 'Islington', 'Kensington and Chelsea', 'Kingston upon Thames', 'Lambeth',
       'Lewisham','Merton', 'Newham', 'Redbridge', 'Richmond upon Thames', 'Southwark', 'Sutton',
       'Tower Hamlets', 'Waltham Forest', 'Wandsworth', 'Westminster']
# Filter the dataframe to include only names in the list:
df = df[df['Borough'].isin(LB_list)]

In [ ]:
#Check which Boroughs are left in the df now:
df['Borough'].unique()

In [ ]:
len(df)

# Group and aggregate
Functions: `groupby()`, `agg()`.

Let's compare the boroughs by how many incidents are recorded for each.

In [ ]:
df.keys() # Same as columns() above.

## groupby()
Which borough has the most crime? We group the data by the "Borough" column, and sum up the total number of cases.

In [ ]:
# Group by Borough, count total number of incidents, sort from highest to lowest
df_grouped = df.groupby(['Borough'])['Crime type'].agg('count').reset_index().sort_values(by=['Crime type'], ascending=False)

In [ ]:
# Show first 10 rows of grouped dataframe
df_grouped

**Task:** Can this data be grouped differently? Let's try it in a separate dataframe (df_grouped2).

In [ ]:
df.keys()

In [ ]:
# Another grouping, by crime type, borough and using "LSOA code" for counting the number of instances.
df_grouped2 = df.groupby(['Crime type', 'Borough'])['LSOA code'].agg('count').reset_index().sort_values(by=['Borough'], ascending=False)
df_grouped2 # display dataframe

In [ ]:
# Here's a selected view showing bicycle thefts by borough, sorted in descending order by frequency of cases
(df_grouped2[df_grouped2['Crime type'].str.contains('bicycle', case=False)]).sort_values(by='LSOA code', ascending=False)

### Simple bar chart comparing the number of cases by borough.

In [ ]:
sns.barplot(data=df_grouped, x="Crime type", y="Borough")

## Crosstab
Compare data across categories with `crosstab()`. A a cross-tabulation table shows the frequency with which certain groups of data appear. 

The Practical Business Python blog has a good summary of Pandas' crosstab function: [Pandas Crosstab Explained](https://pbpython.com/pandas-crosstab.html)

### Simple crosstab
Crime types broken down by Borough.

In [ ]:
df_crossed = pd.crosstab(df['Borough'], df['Crime type']).sort_values(by=['Borough'], ascending=True)

In [ ]:
df_crossed

#### Heatmap of crosstab
The Seaborn `heatmap()` function displays the table as a heatmap.

In [ ]:
sns.heatmap(df_crossed)

**Task:** Would it make sense to show only specific columns? Which ones?

#### Histogram of selected crime type
Shows distribution by crime category.

In [ ]:
sns.displot(df_crossed, x="Violence and sexual offences")

#### Histogram of multiple categories
Here we select multiple columns and instruct Seaborn to display the multiple categories as "stack".

In [ ]:
sns.displot(
    df_crossed[['Anti-social behaviour','Other theft', 'Violence and sexual offences']],
    multiple="stack"
    )

#### Compare frequency of selected crime category across boroughs
First we transform the crosstab dataframe (reindex, sort), and then plot the result:

In [ ]:
df_crossed.keys()

In [ ]:
# Transformation:
# Reset the index for df_crossed to flatten the dataframe
df_crossed.reset_index(inplace=True)
# Name of category to compare (allows us to change category quickly)
crime_category = "Theft from the person"
# Sort the dataframe by the selected category
df_crossed.sort_values(by=[f'{crime_category}'], ascending=False, inplace=True)

Here's the head() showing the top 5 rows for the selected columns (Borough, crime_category):

In [ ]:
df_crossed[['Borough', f'{crime_category}']].head()

In [ ]:
# Display bar plot:
sns.barplot(data=df_crossed, x=f"{crime_category}", y="Borough")

**Task:** Change the crime_cateogory variable above and run the cells again to see how the plot changes.

### Crosstab with margins
The `margins` parameter adds a row at the bottom and column at the end summarising each. The name can be specified with `margins_name`.

In [ ]:
df_crossed_margins = pd.crosstab(df['Borough'], df['Crime type'], margins=True, margins_name='Crimes total').reset_index()

In [ ]:
df_crossed_margins

### Normalise
We can instruct `crosstab()` to show proportions, rather than absolute numbers with the `normalize` parameter.

In [ ]:
# Crosstabl normalised by column, rounding numbers to 2 decimal places
df_cross_pct_cols = pd.crosstab(df['Borough'], df['Crime type'], normalize='columns').round(2)

In [ ]:
df_cross_pct_cols

In [ ]:
# Normalised by rows
df_cross_pct_rows = pd.crosstab(df['Borough'], df['Crime type'], normalize='index').round(2)

In [ ]:
df_cross_pct_rows

#### Histogram of normalised crime types

In [ ]:
sns.displot(df_cross_pct_rows[['Other theft', 'Violence and sexual offences']], element="step")

#### Proportion of crime types by Borough
Heatmap of most frequent crime types for each borough. This enables us to identify "hot spots" within the dataset. The colour palette can be adjusted with the `cmap` parameter (see [Seaborn colour palettes documentation](https://seaborn.pydata.org/tutorial/color_palettes.html)).

In [ ]:
sns.heatmap(df_cross_pct_rows, cmap="mako")

**Task:** Add `margins` to the normalised dataframes, to see proportional distribution between crime types.

# Output
You can save a dataframe in various formats. The most-used methods for writing to file are `to_excel()` and `to_csv()`, and (more simply) copying to clipboard for pasting into Excel or similar software using `to_clipboard()`.

First, list your dataframes, using the `%who_ls` magic method.

In [ ]:
# List all the dataframes in the notebook
%who_ls DataFrame

## Copy to clipboard
The commonly used to_clipboard() method is good for copy/pasting small datasets into Excel (or other spreadsheet software) on the fly.

In [ ]:
# Copy to clipboard
df_cross_pct_rows.to_clipboard()